In [32]:
import pandas as pd

LAYOUT = "../data/all_guis.csv"

In [33]:
layout = pd.read_csv(LAYOUT)

In [48]:
import ast

data_string = layout[layout["id"]==1264]["data"].values[0]

try:
    data = ast.literal_eval(data_string)
    print(data)
except Exception as e:
    print(f"Error parsing string: {e}")

{'bg_color': '#FCFCFA', 'ui_comps': [{'text': 'Instructor', 'resource-id': 'se.perigee.android.seven:id/text1', 'ancestors': ['android.widget.TextView', 'android.view.View', 'java.lang.Object'], 'clickable': False, 'class': 'android.support.v7.widget.AppCompatTextView', 'bounds': [238, 1954, 549, 2051], 'componentLabel': 'Text', 'text_color': '#DB6A58', 'bounds_updated': [244.66666666666666, 1979.3333333333333, 548.6666666666666, 2031.3333333333333], 'text_updated': 'Instructor', 'font_size': 12.765578405174079, 'id': '1264_0'}, {'text': 'Announcer', 'resource-id': 'se.perigee.android.seven:id/text3', 'ancestors': ['android.widget.TextView', 'android.view.View', 'java.lang.Object'], 'clickable': False, 'class': 'android.support.v7.widget.AppCompatTextView', 'bounds': [1036, 1954, 1384, 2051], 'componentLabel': 'Text', 'text_color': '#458786', 'bounds_updated': [1037.3333333333333, 1979.3333333333333, 1384.0, 2031.3333333333333], 'text_updated': 'Announcer', 'font_size': 12.956339739552

In [67]:
#TODO: Dictionary should be different form:
    #Keys -> comps and comp_groups
    #Values -> ids
    #empty groups or groups that only contain on group or component should be deleted. group or component that was inside the deleted group should be elevated one layer

def generate_structure_dict(data):
    # Function to record the deepest level of each component and group
    def record_depth(group, current_depth, component_depths, group_depths):
        group_depths[group['id']] = max(group_depths.get(group['id'], 0), current_depth)
        for comp_id in group.get("ui_comp_ids", []):
            component_depths[comp_id] = max(component_depths.get(comp_id, 0), current_depth)
        for sub_group_id in group.get("ui_comp_group_ids", []):
            sub_group = next((g for g in data["ui_comp_groups"] if g["id"] == sub_group_id), None)
            if sub_group:
                record_depth(sub_group, current_depth + 1, component_depths, group_depths)

    # Function to populate the dictionary at the deepest level they are found
    def populate_dict(group, current_depth, component_depths, group_depths, structure_dict):
        total_items = len(group.get("ui_comp_ids", [])) + len(group.get("ui_comp_group_ids", []))
        
        if group_depths[group['id']] == current_depth and total_items > 1:
            structure_dict[group['id']] = {}

            for comp_id in group.get("ui_comp_ids", []):
                if component_depths[comp_id] == current_depth:
                    structure_dict[group['id']][comp_id] = current_depth

        elif total_items == 1:
            if group.get("ui_comp_ids"):
                structure_dict[group['ui_comp_ids'][0]] = current_depth
            else:
                sub_group_id = group['ui_comp_group_ids'][0]
                sub_group = next(g for g in data["ui_comp_groups"] if g["id"] == sub_group_id)
                populate_dict(sub_group, current_depth, component_depths, group_depths, structure_dict)

        for sub_group_id in group.get("ui_comp_group_ids", []):
            if total_items != 1 or sub_group_id != group['ui_comp_group_ids'][0]:
                sub_group = next((g for g in data["ui_comp_groups"] if g["id"] == sub_group_id), None)
                if sub_group:
                    if group['id'] not in structure_dict:
                        structure_dict[group['id']] = {}
                    populate_dict(sub_group, current_depth + 1, component_depths, group_depths, structure_dict[group['id']])

    component_depths = {}
    group_depths = {}
    structure_dict = {}

    # Record the deepest level of each component and group
    for group in data["ui_comp_groups"]:
        if not any(group['id'] in g.get("ui_comp_group_ids", []) for g in data["ui_comp_groups"]):
            record_depth(group, 1, component_depths, group_depths)

    # Populate the dictionary with components and groups at their deepest level
    for group in data["ui_comp_groups"]:
        if not any(group['id'] in g.get("ui_comp_group_ids", []) for g in data["ui_comp_groups"]):
            populate_dict(group, 1, component_depths, group_depths, structure_dict)

    # Add components that are not in any group to the root level of the structure dictionary
    for component in data["ui_comps"]:
        if component["id"] not in component_depths:
            structure_dict[component["id"]] = 0  # Assuming root level depth as 0

    return structure_dict

# Call the function with your data
structure = generate_structure_dict(data)


In [69]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(structure)

{   '1264_17': 0,
    '1264_21': {   '1264_25': {   '1264_28': {   '1264_31': {   '1264_0': 4,
                                                                '1264_1': 4},
                                                 '1264_6': 3}},
                   '1264_26': {   '1264_29': {   '1264_32': {   '1264_2': 4,
                                                                '1264_3': 4},
                                                 '1264_7': 3}},
                   '1264_27': {   '1264_30': {   '1264_33': {   '1264_4': 4,
                                                                '1264_5': 4},
                                                 '1264_8': 3}},
                   '1264_28': {},
                   '1264_29': {},
                   '1264_30': {}},
    '1264_22': {   '1264_23': {   '1264_34': {'1264_10': 3, '1264_9': 3},
                                  '1264_35': {'1264_11': 3, '1264_12': 3},
                                  '1264_36': {'1264_13': 3, '1264_14': 3}

In [68]:
print(structure)

{'1264_21': {'1264_25': {'1264_28': {'1264_6': 3, '1264_31': {'1264_0': 4, '1264_1': 4}}}, '1264_26': {'1264_29': {'1264_7': 3, '1264_32': {'1264_2': 4, '1264_3': 4}}}, '1264_27': {'1264_30': {'1264_8': 3, '1264_33': {'1264_4': 4, '1264_5': 4}}}, '1264_28': {}, '1264_29': {}, '1264_30': {}}, '1264_22': {'1264_23': {'1264_34': {'1264_9': 3, '1264_10': 3}, '1264_35': {'1264_11': 3, '1264_12': 3}, '1264_36': {'1264_13': 3, '1264_14': 3}, '1264_37': {'1264_15': 3, '1264_16': 3}}}, '1264_24': {'1264_18': 1, '1264_19': 1, '1264_20': 1}, '1264_17': 0}


In [70]:
import json
print(json.dumps(structure, indent=4))

{
    "1264_21": {
        "1264_25": {
            "1264_28": {
                "1264_6": 3,
                "1264_31": {
                    "1264_0": 4,
                    "1264_1": 4
                }
            }
        },
        "1264_26": {
            "1264_29": {
                "1264_7": 3,
                "1264_32": {
                    "1264_2": 4,
                    "1264_3": 4
                }
            }
        },
        "1264_27": {
            "1264_30": {
                "1264_8": 3,
                "1264_33": {
                    "1264_4": 4,
                    "1264_5": 4
                }
            }
        },
        "1264_28": {},
        "1264_29": {},
        "1264_30": {}
    },
    "1264_22": {
        "1264_23": {
            "1264_34": {
                "1264_9": 3,
                "1264_10": 3
            },
            "1264_35": {
                "1264_11": 3,
                "1264_12": 3
            },
            "1264_36": {
          

In [42]:
cangrejo mi tortuga mi animales mis | icon | checkbutton | Skip button | Text Input | Translate this sentence | quit Button 

SyntaxError: invalid syntax (3690680801.py, line 1)